In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
import torch
import torch.nn as nn
from transformers import LxmertTokenizer, LxmertModel
from torchvision import models

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture again as in your original code
class ToxicMemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ToxicMemeClassifier, self).__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        # ResNet backbone
        resnet = models.resnet50(pretrained=True)
        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer
        self.visual_fc = nn.Linear(resnet.fc.in_features, self.lxmert.config.visual_feat_dim)

        # Classifier
        self.fc = nn.Linear(self.lxmert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, images):
        visual_feats = self.cnn_backbone(images).squeeze(-1).squeeze(-1)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)

        batch_size = visual_feats.size(0)
        visual_pos = torch.zeros(batch_size, 1, 4).to(visual_feats.device)

        outputs = self.lxmert(input_ids=input_ids, attention_mask=attention_mask, visual_feats=visual_feats, visual_pos=visual_pos)
        logits = self.fc(outputs.pooled_output)
        return logits

# Initialize the model
model = ToxicMemeClassifier(num_classes=2)

# Load the model weights
model.load_state_dict(torch.load('/content/drive/MyDrive/toxic_meme_classifier.pth'))
model = model.to(device)  # Move model to the right device (CPU or GPU)

# If you want to switch the model to evaluation mode
model.eval()


Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

ToxicMemeClassifier(
  (lxmert): LxmertModel(
    (embeddings): LxmertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LxmertEncoder(
      (visn_fc): LxmertVisualFeatureEncoder(
        (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
        (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (box_fc): Linear(in_features=4, out_features=768, bias=True)
        (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer): ModuleList(
        (0-8): 9 x LxmertLayer(
          (attention): LxmertSelfAttentionLayer(
            (self): LxmertAttention(
              (query):

In [3]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.6/286.6 kB 22.8 MB/s eta 0:00:00


In [11]:
import torch
from PIL import Image
from transformers import LxmertTokenizer
from torchvision import transforms
import easyocr  # EasyOCR library for text extraction

# Define the model architecture again as in your original code
class ToxicMemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ToxicMemeClassifier, self).__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        # ResNet backbone
        resnet = models.resnet50(pretrained=True)
        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer
        self.visual_fc = nn.Linear(resnet.fc.in_features, self.lxmert.config.visual_feat_dim)

        # Classifier
        self.fc = nn.Linear(self.lxmert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, images):
        visual_feats = self.cnn_backbone(images).squeeze(-1).squeeze(-1)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)

        batch_size = visual_feats.size(0)
        visual_pos = torch.zeros(batch_size, 1, 4).to(visual_feats.device)

        outputs = self.lxmert(input_ids=input_ids, attention_mask=attention_mask, visual_feats=visual_feats, visual_pos=visual_pos)
        logits = self.fc(outputs.pooled_output)
        return logits

# Function to load and preprocess a single image
def preprocess_image(img_path, transform):
    image = Image.open(img_path).convert("RGB")
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

# Function to extract text from an image using EasyOCR
def extract_text_from_image(img_path):
    reader = easyocr.Reader(['en'])  # Specify the languages to recognize (English)
    result = reader.readtext(img_path)

    # Combine all the text extracted from the image
    extracted_text = " ".join([text[1] for text in result])  # `text[1]` contains the text part of the result
    return extracted_text.strip()  # Remove any extra spaces/newlines

# Function to tokenize text input
def preprocess_text(text, tokenizer, max_len):
    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0)

# Define the paths for the test image
image_path = "/content/04615.png"  # Replace with your image path

# Load the model and weights
model = ToxicMemeClassifier(num_classes=2)
model.load_state_dict(torch.load('/content/drive/MyDrive/toxic_meme_classifier.pth'))  # Load your trained model
model.to(device)  # Move the model to the appropriate device (CPU or GPU)
model.eval()  # Set model to evaluation mode

# Extract text from the image using EasyOCR
extracted_text = extract_text_from_image(image_path)
print(f"\n\n Extracted Text: {extracted_text}")

# Tokenizer and transform
tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Preprocess the image and text
image = preprocess_image(image_path, transform)
input_ids, attention_mask = preprocess_text(extracted_text, tokenizer, max_len=128)

# Move inputs to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image = image.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Model inference
with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0), image)
    _, prediction = torch.max(outputs, dim=1)

# Output the result
prediction_label = prediction.item()
print(f"Prediction: {'Toxic' if prediction_label == 1 else 'Non-Toxic'}")


Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred



 Extracted Text: @ts the religion 0f peaces join exclusive isam! RESTAP
Prediction: Toxic
